In [1]:
import pandas as pd

In [2]:
# Set display options to show normal numbers with 4 decimal places
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Reload your dataframe to see the changes
df = pd.read_excel("shelby-emissions.xlsx")
df

,State,State-County,Pollutant,Emissions Tons,Pollutant Type,Site Name,EIS Facility ID,Facility Type,Street Address,NAICS,Lat,Lon,EPA Region,FIPS
0,Tennessee,TN-Shelby,Acetaldehyde,13.0790,HAP,"American Yeast Corporation, Tennessee",10693811.0000,Unspecified,2405 North Second Street,All Other Miscellaneous Food Manufacturing,35.1903,-90.0543,4.0000,47157.0000
1,Tennessee,TN-Shelby,Anthracene,0.1590,HAP,ARBOR PRESERVATIVE SYSTEMS LLC,3811011.0000,Unspecified,1471 Channel Ave.,Basic Chemical Manufacturing,35.0994,-90.1008,4.0000,47157.0000
2,Tennessee,TN-Shelby,"Benzo[g,h,i,]Perylene",0.0017,HAP,ARBOR PRESERVATIVE SYSTEMS LLC,3811011.0000,Unspecified,1471 Channel Ave.,Basic Chemical Manufacturing,35.0994,-90.1008,4.0000,47157.0000
3,Tennessee,TN-Shelby,Dibenzofuran,0.0880,HAP,ARBOR PRESERVATIVE SYSTEMS LLC,3811011.0000,Unspecified,1471 Channel Ave.,Basic Chemical Manufacturing,35.0994,-90.1008,4.0000,47157.0000
4,Tennessee,TN-Shelby,Naphthalene,3.5305,HAP,ARBOR PRESERVATIVE SYSTEMS LLC,3811011.0000,Unspecified,1471 Channel Ave.,Basic Chemical Manufacturing,35.0994,-90.1008,4.0000,47157.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371,Tennessee,TN-Shelby,Xylenes (Mixed Isomers),0.1465,HAP,Wm Barr,4143711.0000,Chemical Plant,2105 Channel Avenue/ 2170 Buoy Avenue,All Other Basic Organic Chemical Manufacturing,35.0886,-90.1109,4.0000,47157.0000
1372,Tennessee,TN-Shelby,Chromium (VI),0.0038,HAP,Wright Medical Tech Inc,2847211.0000,Unspecified,5677 Airline Rd.,Surgical Appliance and Supplies Manufacturing,35.2803,-89.6692,4.0000,47157.0000
1373,Tennessee,TN-Shelby,Chromium III,0.0172,HAP,Wright Medical Tech Inc,2847211.0000,Unspecified,5677 Airline Rd.,Surgical Appliance and Supplies Manufacturing,35.2803,-89.6692,4.0000,47157.0000
1374,Tennessee,TN-Shelby,Cobalt,0.0490,HAP,Wright Medical Tech Inc,2847211.0000,Unspecified,5677 Airline Rd.,Surgical Appliance and Supplies Manufacturing,35.2803,-89.6692,4.0000,47157.0000


In [3]:
df_grouped = (
    df.groupby(["Site Name", "Pollutant Type"], as_index=False)
      .agg({
          "Emissions Tons": "sum",
          "Lat": "first",
          "Lon": "first"
      })
)
df_grouped

,Site Name,Pollutant Type,Emissions Tons,Lat,Lon
0,ARBOR PRESERVATIVE SYSTEMS LLC,HAP,7.9245,35.0994,-90.1008
1,"American Yeast Corporation, Tennessee",HAP,13.0790,35.1903,-90.0543
2,Atofina Chemicals Inc,CAP,132.7699,35.2677,-89.9775
3,Atofina Chemicals Inc,Other,5.1048,35.2677,-89.9775
4,BLUES CITY BREWERY LLC,CAP,120.9069,35.0344,-89.8889
...,...,...,...,...,...
175,"Valero Refining Co. (Prev. Premcor Refining, P...",GHG,1295666.0907,35.0853,-90.0837
176,"Valero Refining Co. (Prev. Premcor Refining, P...",HAP,77.9705,35.0853,-90.0837
177,"Valero Refining Co. (Prev. Premcor Refining, P...",Other,250.1832,35.0853,-90.0837
178,Wm Barr,HAP,2.6720,35.0886,-90.1109


In [13]:
# Pivot the DataFrame to create wide format
df_wide = df_grouped.pivot(
    index=['Site Name', 'Lat', 'Lon'],
    columns='Pollutant Type',
    values='Emissions Tons'
).reset_index()

# Fill NaN values with 0
df_wide = df_wide.fillna(0)

# Add total emissions column
df_wide['Total Emissions'] = df_wide[['CAP', 'GHG', 'HAP', 'Other']].sum(axis=1)

# Display the result
df_wide['Site Name'].count()

np.int64(60)

In [5]:
df = df_wide

In [6]:
df.columns

Index(['Site Name', 'Lat', 'Lon', 'CAP', 'CAP/HAP', 'GHG', 'HAP', 'Other',
       'Total Emissions'],
      dtype='object', name='Pollutant Type')

In [11]:
new_df = df.sort_values(by='Total Emissions', ascending=False).head(20)
new_df

Pollutant Type,Site Name,Lat,Lon,CAP,CAP/HAP,GHG,HAP,Other,Total Emissions
53,TVA-Allen Fossil Plant,35.0741,-90.1488,0.5955,0.0000,1467191.1085,0.0003,0.0055,1467191.7098
57,"Valero Refining Co. (Prev. Premcor Refining, P...",35.0853,-90.0837,2169.6621,0.0006,1295666.0907,77.9705,250.1832,1298163.9065
31,Memphis Intl,35.0385,-89.9781,4257.0009,0.0350,305269.5222,96.3015,18.8379,309641.6625
36,Nucor Steel Memphis Inc.,35.0486,-90.1644,1345.9460,0.1230,155153.4863,0.5430,88.8973,156588.8725
40,PMC Biogenix (formerly Crompton Corporation/ E...,35.1665,-89.9599,611.6871,0.0006,108041.3366,3.8072,11.9006,108668.7313
26,Lucite International Inc,35.2708,-89.9751,0.0000,0.0000,66616.1606,62.8195,0.0000,66678.9801
8,Cargill Corn Milling,35.0818,-90.1339,389.2144,0.0000,28279.2945,4.8353,27.5081,28700.8523
5,Buckeye Technologies Inc,35.1575,-89.9628,129.9201,0.0001,26706.1482,3.4223,7.8543,26847.3450
45,Solae LLC,35.0313,-89.8822,453.1891,0.0010,17705.6481,1.3526,36.2236,18196.4134
30,Memphis,35.1229,-90.0179,240.8067,0.0000,9733.5194,5.4620,14.7841,9994.5722


In [12]:
new_df['Total Emissions'].count()

np.int64(20)